In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pandas_profiling
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('data/aac_shelter_outcomes.csv')
data.head(5)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


In [3]:
data.shape

(78256, 12)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78256 entries, 0 to 78255
Data columns (total 12 columns):
age_upon_outcome    78248 non-null object
animal_id           78256 non-null object
animal_type         78256 non-null object
breed               78256 non-null object
color               78256 non-null object
date_of_birth       78256 non-null object
datetime            78256 non-null object
monthyear           78256 non-null object
name                54370 non-null object
outcome_subtype     35963 non-null object
outcome_type        78244 non-null object
sex_upon_outcome    78254 non-null object
dtypes: object(12)
memory usage: 7.2+ MB


1. Нет дискретных данных - все данные или строковые или дата 
2. outcome_subtype бесполезный - так как является подтипом значения кооторое предстоит предсказывать
3. datetime и monthyear идентичны ???
4. age_upon_outcome неудобная для анализа - возраст при поступлении
5. есть смысл сделать колонку возраст при поступлении ? 
6. animal_id внутренний идентификатор и для предсказания нам не пригодится
7. В именах животных есть аномалии - есть смысл удалить все символы лишние кроме букв

In [5]:
data['color'].value_counts()

Black/White               8153
Black                     6602
Brown Tabby               4445
Brown                     3486
White                     2784
                          ... 
Liver/Brown Brindle          1
Cream/Red Tick               1
Lynx Point/Brown Tabby       1
Silver Tabby/Black           1
Gray/Tricolor                1
Name: color, Length: 525, dtype: int64

In [6]:
# проверяем что колонки идентичны
data[data['datetime']!=data['monthyear']].shape


(0, 12)

In [11]:
data.drop(['age_upon_outcome', 'monthyear', 'outcome_subtype','animal_id'], axis=1, inplace=True)

KeyError: "['age_upon_outcome' 'monthyear' 'outcome_subtype' 'animal_id'] not found in axis"

In [12]:
data.head(5)

,animal_type,breed,color,date_of_birth,datetime,name,outcome_type,sex_upon_outcome
0,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,NaN,Transfer,Intact Male
1,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,Lucy,Transfer,Spayed Female
2,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,*Johnny,Adoption,Neutered Male
3,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,Monday,Transfer,Neutered Male
4,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,NaN,Euthanasia,Unknown


In [13]:
data.color = data.color.str.lower()

In [14]:
data['name'].value_counts()

Bella           344
Max             319
Luna            239
Rocky           236
Princess        235
               ... 
Ferbie            1
Buddy Henry       1
*Cosima           1
Michaelscott      1
*Zale             1
Name: name, Length: 14574, dtype: int64

In [15]:
# смотрим где вместо имени NaN
data[data['name'].isna()]

,animal_type,breed,color,date_of_birth,datetime,name,outcome_type,sex_upon_outcome
0,Cat,Domestic Shorthair Mix,orange tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,NaN,Transfer,Intact Male
4,Other,Bat Mix,brown,2014-01-07T00:00:00,2014-07-07T14:04:00,NaN,Euthanasia,Unknown
6,Other,Squirrel Mix,tan,2013-12-13T00:00:00,2014-12-13T12:20:00,NaN,Euthanasia,Unknown
17,Cat,Domestic Shorthair Mix,black/white,2013-12-16T00:00:00,2014-01-09T19:29:00,NaN,Transfer,Intact Male
20,Cat,Domestic Shorthair Mix,brown tabby/white,2013-07-05T00:00:00,2014-03-06T14:29:00,NaN,Transfer,Unknown
...,...,...,...,...,...,...,...,...
78250,Dog,Golden Retriever/Labrador Retriever,brown/white,2017-12-04T00:00:00,2018-02-01T18:40:00,NaN,Adoption,Neutered Male
78251,Dog,Golden Retriever/Labrador Retriever,brown/white,2017-12-04T00:00:00,2018-02-01T18:26:00,NaN,Adoption,Spayed Female
78253,Other,Bat Mix,brown,2017-02-01T00:00:00,2018-02-01T18:08:00,NaN,Euthanasia,Unknown
78254,Dog,Standard Schnauzer,red,2017-11-13T00:00:00,2018-02-01T18:32:00,NaN,Adoption,Spayed Female


In [16]:
#проставляем признак - нет имени
data['name'].fillna("noname", inplace = True)

In [17]:
# чистим имя - убираем звездочки и прочий мусор
import re
data['name'] = data['name'].map(lambda x: re.sub(r'[^A-Za-z]', '', x))

In [18]:
# Преобразуем дату рождения в удобный формат
import datetime as DT

data['date_of_birth'] = data['date_of_birth'].map(lambda x: x[:10])
data['datetime'] = data['datetime'].map(lambda x: x[:10])
# вводим новый атрибут - возраст в днях на момент выхода вместо age_upon_outcome
data['age_days_out'] = data.apply(lambda x: (DT.datetime.strptime(x['datetime'], '%Y-%m-%d') - DT.datetime.strptime(x['date_of_birth'], '%Y-%m-%d')).days, axis = 1)

data.head(5)

,animal_type,breed,color,date_of_birth,datetime,name,outcome_type,sex_upon_outcome,age_days_out
0,Cat,Domestic Shorthair Mix,orange tabby,2014-07-07,2014-07-22,noname,Transfer,Intact Male,15
1,Dog,Beagle Mix,white/brown,2012-11-06,2013-11-07,Lucy,Transfer,Spayed Female,366
2,Dog,Pit Bull,blue/white,2013-03-31,2014-06-03,Johnny,Adoption,Neutered Male,429
3,Dog,Miniature Schnauzer Mix,white,2005-06-02,2014-06-15,Monday,Transfer,Neutered Male,3300
4,Other,Bat Mix,brown,2014-01-07,2014-07-07,noname,Euthanasia,Unknown,181


In [19]:
# колонка datetime формально теперь не нужна
# одновременно есть гипотеза что на показатель outcome_type Adoption - что животное кто то забрал, и Transfer влияет месяц или день недели
# делаем колонки с годом, месяцем, днем недели от 1 до 7 и дропаем дату
data['year_out'] = data.apply(lambda x: DT.datetime.strptime(x['datetime'], '%Y-%m-%d').year, axis = 1)
data['month_out'] = data.apply(lambda x: DT.datetime.strptime(x['datetime'], '%Y-%m-%d').month, axis = 1)
data['week_day_out'] = data.apply(lambda x: DT.datetime.strptime(x['datetime'], '%Y-%m-%d').isoweekday(), axis = 1)
data.drop(['datetime'], axis=1, inplace=True)

In [20]:
data.head(5)

,animal_type,breed,color,date_of_birth,name,outcome_type,sex_upon_outcome,age_days_out,year_out,month_out,week_day_out
0,Cat,Domestic Shorthair Mix,orange tabby,2014-07-07,noname,Transfer,Intact Male,15,2014,7,2
1,Dog,Beagle Mix,white/brown,2012-11-06,Lucy,Transfer,Spayed Female,366,2013,11,4
2,Dog,Pit Bull,blue/white,2013-03-31,Johnny,Adoption,Neutered Male,429,2014,6,2
3,Dog,Miniature Schnauzer Mix,white,2005-06-02,Monday,Transfer,Neutered Male,3300,2014,6,7
4,Other,Bat Mix,brown,2014-01-07,noname,Euthanasia,Unknown,181,2014,7,1


In [21]:
data['month_out'].value_counts()

10    7773
7     7435
6     7013
11    6977
12    6957
8     6840
5     6832
9     6422
1     6321
3     5488
4     5467
2     4731
Name: month_out, dtype: int64

In [22]:
data['year_out'].value_counts()

2014    18709
2015    18506
2016    17671
2017    17659
2013     4505
2018     1206
Name: year_out, dtype: int64

6    12583
7    12163
2    11456
1    11226
5    10482
3    10295
4    10051
Name: week_day_out, dtype: int64

In [24]:

data['sex_upon_outcome'].value_counts()

Neutered Male    27784
Spayed Female    25203
Intact Male       9549
Intact Female     9143
Unknown           6575
Name: sex_upon_outcome, dtype: int64

In [45]:
pandas_profiling.ProfileReport(data)

D:\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,11
Number of observations,78256
Total Missing (%),0.0%
Total size in memory,6.6 MiB
Average record size in memory,88.0 B
Numeric,4
Categorical,7
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [44]:
# Цвета содержат аномалии - перемена мест слагаемых
data['color'] = data['color'].map(lambda x: x if x.find('/')==-1 else ('/'.join(sorted(x.split('/')))))

In [119]:
# оставляем записи только с теми критериями которые нам нужны
df = data[data.outcome_type.isin(['Adoption','Transfer'])]
df.shape

(56611, 11)

In [120]:
# добавляем целевые признаки 1 - если забрали люди из приюта и 0 если трансфер- нужно ли это?
df['res'] = df.apply(lambda x: 1 if x['outcome_type'] == 'Adoption' else 0, axis = 1)
df.head(10)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,animal_type,breed,color,date_of_birth,name,outcome_type,sex_upon_outcome,age_days_out,year_out,month_out,week_day_out,res
0,Cat,Domestic Shorthair Mix,orange tabby,2014-07-07,noname,Transfer,Intact Male,15,2014,7,2,0
1,Dog,Beagle Mix,brown/white,2012-11-06,Lucy,Transfer,Spayed Female,366,2013,11,4,0
2,Dog,Pit Bull,blue/white,2013-03-31,Johnny,Adoption,Neutered Male,429,2014,6,2,1
3,Dog,Miniature Schnauzer Mix,white,2005-06-02,Monday,Transfer,Neutered Male,3300,2014,6,7,0
5,Dog,Leonberger Mix,brown/white,2013-06-03,Edgar,Transfer,Intact Male,126,2013,10,1,0
7,Dog,Chihuahua Shorthair Mix,brown,2011-11-23,Ella,Transfer,Spayed Female,1111,2014,12,1,0
8,Cat,Domestic Shorthair Mix,blue tabby/white,2014-06-16,Lucy,Adoption,Intact Female,59,2014,8,4,1
9,Cat,Domestic Shorthair Mix,black/white,2014-03-26,Frida,Adoption,Spayed Female,95,2014,6,7,1
11,Dog,Papillon/Border Collie,black/white,2012-02-28,Fancy,Transfer,Neutered Male,759,2014,3,5,0
12,Dog,Chihuahua Shorthair/Pomeranian,black,2014-03-07,Kash,Adoption,Neutered Male,80,2014,5,1,1


In [121]:
pandas_profiling.ProfileReport(df)

D:\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,13
Number of observations,56611
Total Missing (%),0.0%
Total size in memory,5.6 MiB
Average record size in memory,104.0 B
Numeric,5
Categorical,7
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [67]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

X, y = df.drop(['outcome_type'], axis=1), df.outcome_type 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [87]:
# Кодируем типы животных
#pd.get_dummies(df['animal_type']) 
#df.head(10)

,animal_type,breed,color,date_of_birth,name,outcome_type,sex_upon_outcome,age_days_out,year_out,month_out,week_day_out,res
0,1,Domestic Shorthair Mix,orange tabby,2014-07-07,noname,Transfer,Intact Male,15,2014,7,2,0
1,2,Beagle Mix,brown/white,2012-11-06,Lucy,Transfer,Spayed Female,366,2013,11,4,0
2,2,Pit Bull,blue/white,2013-03-31,Johnny,Adoption,Neutered Male,429,2014,6,2,1
3,2,Miniature Schnauzer Mix,white,2005-06-02,Monday,Transfer,Neutered Male,3300,2014,6,7,0
5,2,Leonberger Mix,brown/white,2013-06-03,Edgar,Transfer,Intact Male,126,2013,10,1,0
7,2,Chihuahua Shorthair Mix,brown,2011-11-23,Ella,Transfer,Spayed Female,1111,2014,12,1,0
8,1,Domestic Shorthair Mix,blue tabby/white,2014-06-16,Lucy,Adoption,Intact Female,59,2014,8,4,1
9,1,Domestic Shorthair Mix,black/white,2014-03-26,Frida,Adoption,Spayed Female,95,2014,6,7,1
11,2,Papillon/Border Collie,black/white,2012-02-28,Fancy,Transfer,Neutered Male,759,2014,3,5,0
12,2,Chihuahua Shorthair/Pomeranian,black,2014-03-07,Kash,Adoption,Neutered Male,80,2014,5,1,1


In [86]:
df.head(10)

,animal_type,breed,color,date_of_birth,name,outcome_type,sex_upon_outcome,age_days_out,year_out,month_out,week_day_out,res
0,1,Domestic Shorthair Mix,orange tabby,2014-07-07,noname,Transfer,Intact Male,15,2014,7,2,0
1,2,Beagle Mix,brown/white,2012-11-06,Lucy,Transfer,Spayed Female,366,2013,11,4,0
2,2,Pit Bull,blue/white,2013-03-31,Johnny,Adoption,Neutered Male,429,2014,6,2,1
3,2,Miniature Schnauzer Mix,white,2005-06-02,Monday,Transfer,Neutered Male,3300,2014,6,7,0
5,2,Leonberger Mix,brown/white,2013-06-03,Edgar,Transfer,Intact Male,126,2013,10,1,0
7,2,Chihuahua Shorthair Mix,brown,2011-11-23,Ella,Transfer,Spayed Female,1111,2014,12,1,0
8,1,Domestic Shorthair Mix,blue tabby/white,2014-06-16,Lucy,Adoption,Intact Female,59,2014,8,4,1
9,1,Domestic Shorthair Mix,black/white,2014-03-26,Frida,Adoption,Spayed Female,95,2014,6,7,1
11,2,Papillon/Border Collie,black/white,2012-02-28,Fancy,Transfer,Neutered Male,759,2014,3,5,0
12,2,Chihuahua Shorthair/Pomeranian,black,2014-03-07,Kash,Adoption,Neutered Male,80,2014,5,1,1


In [113]:
# предположи что на резульатт может влиять чистая порода или смешанная
df[df['breed'].str.contains('Mix')]['breed'].shape


(56611, 12)

In [114]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
label_encoder = LabelEncoder()   
# Кодируем колонку animal_type в цифры
df['animal_type']= label_encoder.fit_transform(df['animal_type'])
df['sex_upon_outcome']= label_encoder.fit_transform(df['sex_upon_outcome'])

df.head()


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,animal_type,breed,color,date_of_birth,name,outcome_type,sex_upon_outcome,age_days_out,year_out,month_out,week_day_out,res
0,1,Domestic Shorthair Mix,orange tabby,2014-07-07,noname,Transfer,1,15,2014,7,2,0
1,2,Beagle Mix,brown/white,2012-11-06,Lucy,Transfer,3,366,2013,11,4,0
2,2,Pit Bull,blue/white,2013-03-31,Johnny,Adoption,2,429,2014,6,2,1
3,2,Miniature Schnauzer Mix,white,2005-06-02,Monday,Transfer,2,3300,2014,6,7,0
5,2,Leonberger Mix,brown/white,2013-06-03,Edgar,Transfer,1,126,2013,10,1,0


In [122]:
ohe = OneHotEncoder(categories = ['animal_type'])  
print(ohe)
df = ohe.fit_transform(df).toarray()

OneHotEncoder(categorical_features=None, categories=['animal_type'], drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)


ValueError: Shape mismatch: if categories is an array, it has to be of shape (n_features,).

In [70]:
from sklearn.ensemble import RandomForestClassifier

# создаем модель деревья решений
# выбираем 100 деревьев в качестве параметра
model=RandomForestClassifier(n_estimators=100)

# обучаем модель
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'Cat'